# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''

'\nYour code here\n\n提示：可參考以下的Stack Overflow: \nhttps://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166\n'

In [19]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, 100000);")
    print("網頁更新中")
    time.sleep(2)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    new_height =  browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    if new_height == last_height:
        print('stop')
        browser.close()
        break


網頁更新中
stop


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [17]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [18]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 永豐金董事候選人名單出列　何家人退位　女性擔任董座呼聲高

▲永豐金控。（圖／記者湯興漢攝）
記者吳靜君／台北報導
永豐金今年將董監改選，為了因應金管會要求，希望可以降低家族色彩，永豐金今年董事提名名單，何家人全數退出，包含何壽川之子何奕達、前財政部長邱正雄與現任董事長翁文祺等，至於新任董事名單由身為女性的陳思寬呼聲高，可望成為首位金控女董座。
永豐金預計在5月召開董事會，為了防止寶佳進入董事會，永豐金決議將這一屆的董事席次從原本的9席縮減為7席。
永豐金今日也公告新的董事候選人名單上誼文化陳思寬、朱士廷、行遠投資的曹為實、葉奇鑫；獨立董事則是證交所前董事長薛琦、中信金風控長許建基、東吳大學潘維大。
值得注意的是，何家人何奕達以及被視為何家陣營的邱正雄與翁文祺等都不在名單之內。
若從名單來看，必須從四位一般董事選出董事長，據瞭解，陳思寬以女性董事及專精總體經濟、貨幣政策出任，非常有機會出任董事長。但永豐金強調，結果還必須通過股東會選舉及董事會最後決定。
----------------------------------------------------------------------

[1] 經長糾永豐餘、正隆拍片喊話「囤貨潮」　衛生紙足夠「七卡臣300年」

▲經濟部長沈榮津今拍影片指出，包括衛生紙、泡麵、罐頭廠商都已全力製造加大產能。（圖／經濟部提供）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情延燒，全台掀起囤貨潮亂象，經濟部長沈榮津今（20）日糾衛生紙大廠永豐餘、正隆春風等業者拍片，親自走訪衛生紙廠，帶國人一探衛生紙產線，「存量足夠七卡臣300年」，向國人喊話貨源供應無虞、不必搶購。
經濟部長沈榮津今拍影片指出，包括衛生紙、泡麵、罐頭廠商都已全力製造加大產能。沈榮津說，實地參訪永豐餘及正隆春風等廠商，衛生紙原料、供貨無虞，也已24小時生產；泡麵廠維力、味王產能「已拉高到中元普渡時的水準」，罐頭廠商也全力生產，目前已有三個月存量。
新冠肺炎疫情升溫，民眾因預期心理爆發囤貨潮，造成全台賣場通路排隊結帳等亂象，經濟部長沈榮津今拍影片親自走訪衛生紙廠；另外行政院長蘇貞昌鼓勵民眾「儘量買」，強調貨源

聯準會連續兩次突然性降息給市場下足猛藥，希望能提振市場信心。從金融海嘯的經驗來看，聯準會把市場利率降到底之後還需要一段期間市場才會落底。
文／魏聖峰
聯準會再度突襲！就在聯準會即將召開三月例行性ＦＯＭＣ貨幣政策會議前夕，主席鮑威爾在十六日亞洲股市開盤前夕宣布降息四碼，同時加碼七千億美元的ＱＥ政策。聯準會的突襲與三月三日美股盤中宣布的降息如出一轍，結果造成美股三大期指電子盤全部下跌近五％，並再度出現熔斷機制。十六日的亞洲股市全面下挫，市場哀鴻遍野。
降息還無法扭轉跌勢
美股和全球股市從二月下旬至今，受到新冠肺炎疫情在歐美各國擴大，衝擊當地消費市場，造成全球股市重挫。這波全球股市遭到空頭摜壓，把美股從二○○九年三月至今長達十一年的多頭市場遭到扭轉成為熊市，全球股市也從從多頭市場轉為空頭市場。這次聯準會的降息動作還和其他國際五大央行，包括歐洲央行、日本央行、加拿大央行和瑞士央行聯手，調降換匯利率二五個基點（一碼），六大央行將利用現有貨幣交換額度在國際市場相互支援，以確保美元的流動性穩健。紓解在全球疫情期間全球支付系統吃緊，滿足全球企業和金融機構應急美元之用。這次聯準會和全球央行在金融海嘯之後首度的聯合行動也因應市場的危機。
聯準會原本要在三月十七、十八日召開例行性ＦＯＭＣ會議，突然提前四天宣布最新的利率政策，且聯準會這回下的猛藥來得又凶又急，不但降息的幅度遠超過市場預期的兩碼，還擴大新的ＱＥ政策。
聯準會和其他五大央行的聯手行動並沒有讓美股和全球股市馬上翻轉，市場並以利空來解讀這樣的動作。市場擔心的是，新冠肺炎疫情在歐美各國仍在擴大階段還沒有像中國那樣疫情已從高點緩和。降息和擴大ＱＥ政策只能提振市場的信心，最重要的那顆解決新冠肺炎的有效藥還沒出現，這才是市場現階段最擔心的問題。即便如此，聯準會這時候還是得降息和ＱＥ，現階段政策暫時無效並不代表未來會沒用，聯準會該做的事還是得要做，外加提供市場貼現窗口與回購隔夜拆款資金，未來還可能有來自財政部的振興政策來扭轉市場的趨勢。
放眼聯準會從二○○四年以來的貨幣政策動作，已經出現兩輪的升息和降息循環，利率回到○～○．二五％。放眼聯準會在二○○八年九月間發生的信貸危機並引發後來的全球股市崩盤的金融海嘯，當年一輪降息首度把利率降到和現在一樣的零利率水準，這段期間聯準會共出現三次在非ＦＯＭＣ貨幣政策會議期間的降息，當年聯準會也曾經和全球大

▲勞動部。（圖／記者余弦妙攝）
記者余弦妙／台北報導
新冠肺炎疫情持續延燒，印尼勞動部指出20日起將暫時停止輸出所有移工，目的在於保護國內外移工。對此，我國勞動部則鼓勵雇主採續聘及國內承接維持移工人數，這樣也能防止人員往來出現防疫破口。
勞動部跨國勞動力管理組組長薛鑑忠表示，目前在台印尼移工有27.6萬人，占了總外籍移工的38％，27.6萬人中看護工就有20萬，佔了約70％；外籍漁工1.2萬人，其中印尼籍約9000多人，同樣佔了約70%；另外43.9萬外籍製造業移工中，印尼籍6.5萬人約佔了15％。
雖然目前印尼禁止移工輸出，不過薛鑑忠也說，目前台灣還有泰國、越南、菲律賓等國家的移工可以輸入，因此目前除了鼓勵雇主繼續續聘或國內承接原本印尼移工外，雇主也能選擇聘僱其他國籍的移工。
----------------------------------------------------------------------

[22] 富士達保經董事長廖學茂「實務教學獲師生肯定」　晉升朝陽科大講座教授

▲富士達保險經紀人董事長廖學茂。（圖／富士達保經提供）
記者楊絡懸／台北報導
富士達保險經紀人董事長廖學茂通過朝陽科技大學校方審核，肯定其專業能力與教學品質，晉級升等管理學院「學門講座教授」。廖學茂說，將繼續傳授財務金融、人力資源管理和行銷等結合實務的學術知識，幫助莘莘學子求學期間奠定專業基礎，逐步了解產業趨勢，也期盼學生們畢業後，其所學也能與業界現況接軌。
廖學茂踏入保險業30多年，從基層業務員做起，培養保險銷售技巧，爾後一路表現優異獲得讚賞，先後於數家大型保險公司擔任過執行副總經理、總經理等高階主管職，期間也不斷精進自身企業經營管理能力。
2007年，廖學茂看見保險業產銷分離的趨勢，毅然成立富士達保險經紀人公司，在其嚴謹領導下，富士達保經目前更擠身台灣前十大保經公司，旗下多達2500多名同仁。
擁有豐富且完整的行銷經驗和企業管理實務，廖學茂也投身教育，曾擔任東南科大和崇右科大助理教授，以及政大商學院企業導師，傳授自身經驗協助產學合作培育。
2010年起，朝陽科大希望借重專業及豐富的業界人脈和社會聲望，正式邀請廖學茂於「行銷與流通管理系」任教。十多年的教學經驗，孕育不少優秀學子，廖學茂也獲得許多寶貴回饋，且應用至公司經營上，讓專案制度更加友善新鮮人並提供完善教育訓練，充

▲宏達電新店總部。（圖／記者湯興漢攝）
記者姚惠茹／台北報導
宏達電（2498）今（20）日公布2019年第4季營收18.1億元，毛利率25.7%，較上季的22.9%，增加2.8個百分點，創2017年第4季以來，連續8個季度毛利率成長，稅後淨損23.7億元，每股稅後虧損2.87元。
回顧去年第4季營運，宏達電表示，VIVE虛擬實境方面，VIVE Cosmos年底在全球開賣，作為最新款高規電腦驅動PC VR產品，使用者不再需要安裝定位基地台，就能享受便利且絕佳的虛擬實境體驗，同時可換式面板模組設計可隨著使用者的需求，調整或擴充頭戴式顯示器的功能。
藝術領域應用方面，宏達電分享，攜手法國羅浮宮在達文西逝世500週年紀念特展中推出虛擬實境作品《蒙娜麗莎：越界視野》，觀眾可透過配戴VIVE Cosmos體驗，這也是達文西位於羅浮宮的名作《蒙娜麗莎》的第一個虛擬實境內容。
宏達電指出，VIVEPORT平台也在第4季推出全球首款無線跨平台內容串流模式，VIVEPORT串流Beta版服務，支援VIVE Focus Plus頭戴式顯示器，實現PC端的VR優質內容無縫串流至一體機VR虛擬實境裝置，並讓VIVEPORT Infinity串流功能可同時支援Oculus Link Beta版。
手機業務方面，宏達電表示，去年第4季推出全球首款支援完整比特幣節點的智慧型手機EXODUS 1s，這也是EXODUS 1的平價版本，並將金融服務移轉至個人手中，使用者能在手機上進行比特幣交易服務，並陸續在歐洲、台灣、沙烏地阿拉伯和阿拉伯聯合大公國推出。
宏達電表示，HTC也在去年底推出Desire 19s手機，延續Desire系列的經典設計，具高性價比、6.2吋極致窄邊大螢幕、超大容量3850mAh電池與超廣角三鏡，主相機搭配高階HDR與能拍出專業及人像照的1600萬像素的前置相機，是中階手機的最佳選擇。
----------------------------------------------------------------------

[35] 聯華攜手鼎泰豐麵粉供應商　僑泰興強強聯手「不排除上市」

▲聯華實業目前是全台規模最大、產量最高的老字號麵粉廠，能依照客戶生產製程與產品特性，客製化超過150種以上的專用粉。圖為聯華投控總經理林信宏。（圖／記者林淑慧攝）
記者林淑慧／台北報導
全